<a href="https://colab.research.google.com/github/rodezi/EntregableBEDU_Analisis_Equipo12_ProyectoFinal/blob/main/Proyecto_Final_BEDU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equipo 12.

## Proyecto Final BEDU
En el siguiente proyecto llevo a cabo un analisis exploratorio de datos y posterior al analisis, un modelo de regresion logistica.

Para este proyecto se uso el data set "Anti - Money Laundering" de Kaggle.
https://www.kaggle.com/datasets/berkanoztas/synthetic-transaction-monitoring-dataset-aml

Para correr el analisis hay que descargar el data set y cargarlo en el icono de "carpeta" que se encuentra a lado izquierdo, el data set aparecera como "SAML-D"

Se cargan las librerias a utilizar en el proyecto

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl

## Carga del data set

In [2]:
df = pd.read_csv("SAML-D.csv")

Informacion sobre el data set, que nos muestra registros y columnas.
NO hay registros nulos

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8754485 entries, 0 to 8754484
Data columns (total 12 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Time                    object 
 1   Date                    object 
 2   Sender_account          int64  
 3   Receiver_account        int64  
 4   Amount                  float64
 5   Payment_currency        object 
 6   Received_currency       object 
 7   Sender_bank_location    object 
 8   Receiver_bank_location  object 
 9   Payment_type            object 
 10  Is_laundering           int64  
 11  Laundering_type         object 
dtypes: float64(1), int64(3), object(8)
memory usage: 801.5+ MB


Aqui elimine las columnas **"Time"** y **"Date"**.

In [4]:
df.drop(['Time', 'Date'], axis=1, inplace=True)

Aqui se hace una visualizacion de columnas como: **Payment currency, Sender Bank location, Payment Type**. entre otras. Nos dio datos valiosos sobre como determinar el proyecto a continuacion.

In [5]:
for col in ['Payment_currency', 'Received_currency', 'Sender_bank_location',
            'Receiver_bank_location', 'Payment_type', 'Laundering_type']:
    print(f"Columna: {col}")
    print(df[col].value_counts())
    print("\n" + "="*50 + "\n")

Columna: Payment_currency
Payment_currency
UK pounds          8382986
Euro                107799
Dirham               25491
Swiss franc          25279
Pakistani rupee      24971
Turkish lira         24939
Naira                24801
US dollar            23922
Yen                  23489
Indian rupee         22983
Moroccan dirham      22980
Albanian lek         22474
Mexican Peso         22371
Name: count, dtype: int64


Columna: Received_currency
Received_currency
UK pounds          8089163
Euro                213926
Pakistani rupee      42590
Moroccan dirham      42576
Mexican Peso         42076
Albanian lek         41879
Yen                  41840
Naira                41521
Indian rupee         40697
US dollar            39793
Dirham               39648
Swiss franc          39451
Turkish lira         39325
Name: count, dtype: int64


Columna: Sender_bank_location
Sender_bank_location
UK             8460130
UAE              18896
Switzerland      18873
Pakistan         18642
Turkey     

Aqui hice una conversion de las tasas de cambio a dolares y una nueva columna con el nuevo valor (USD)

In [6]:
# Tasas de cambio hacia USD (example rates, you may need to adjust based on the data's timeframe)
conversion_to_usd = {
    'UK POUNDS': 1.28,
    'EURO': 1.16,
    'DIRHAM': 0.27,
    'SWISS FRANC': 1.11,
    'PAKISTANI RUPEE': 0.0036,
    'TURKISH LIRA': 0.037,
    'NAIRA': 0.00067,
    'US DOLLAR': 1.0,
    'YEN': 0.0068,
    'INDIAN RUPEE': 0.012,
    'MOROCCAN DIRHAM': 0.10,
    'ALBANIAN LEK': 0.011,
    'MEXICAN PESO': 0.059
}

# Crear nueva columna en USD
def convertir_a_usd(row):
    rate = conversion_to_usd.get(row['Payment_currency'], 1.0)
    return row['Amount'] * rate

df['Amount_usd'] = df.apply(convertir_a_usd, axis=1)

display(df[['Amount', 'Payment_currency', 'Amount_usd']].head(10))


,Amount,Payment_currency,Amount_usd
0,1459.15,UK pounds,1459.15
1,6019.64,UK pounds,6019.64
2,14328.44,UK pounds,14328.44
3,11895.00,UK pounds,11895.00
4,115.25,UK pounds,115.25
5,5130.99,UK pounds,5130.99
6,12176.52,UK pounds,12176.52
7,56.90,UK pounds,56.90
8,4738.45,UK pounds,4738.45
9,5883.87,Indian rupee,5883.87


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8754485 entries, 0 to 8754484
Data columns (total 11 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Sender_account          int64  
 1   Receiver_account        int64  
 2   Amount                  float64
 3   Payment_currency        object 
 4   Received_currency       object 
 5   Sender_bank_location    object 
 6   Receiver_bank_location  object 
 7   Payment_type            object 
 8   Is_laundering           int64  
 9   Laundering_type         object 
 10  Amount_usd              float64
dtypes: float64(2), int64(3), object(6)
memory usage: 734.7+ MB


## Aqui empieza mi analisis exploratorio de datos.

Numero total de transacciones:

Proporción de transacciones legítimas y sospechosas (lavado de dinero):

*   Legitimas: 8,745,523
*   Fraudulentas: 8, 962

Transacciones por tipo de método (ACH, crédito, débito, cheque):

**Las cuatro principales**

*   Tarjeta de credito: 1,853,835
*   Tarjeta de debito: 1,853,199
*   Cheque: 1,852,629
*   ACH: 1,850,212

**Distribución de montos**

¿Cuál es la distribución de los montos de transacción?

¿Las transacciones sospechosas tienden a tener montos más altos que las legítimas?

Si, la media de transaciones sospechosas es de 41,302 usd

Mientras que la de transaciones normales es de 8, 736 usd

De donde vienen las transacciones mas sospechosas?

*  Uk
*  Marruecos
*  Mexico
*  India
*  Suiza

¿Cuál es el monto promedio y mediano por tipo de transacción?

* Legitimo: 41,302
* Sospechoso: 8, 736

¿Qué tipo de método tiene mayor proporción de transacciones sospechosas?

Los primeros tres en la lista son:

*  Cross-border
*  Cash deposit
*  Cash Withdrawal

Los metodos que se hacen via **ACH** tienen una mayor probabilidad de ser fraudulentos.



In [11]:
total_transactions = int(len(df))
print(f"Total number of transactions: {total_transactions:,}")

Total number of transactions: 8,754,485


In [12]:
def format_number(number):
    return f'{number:,}'

legit_transactions = df[df['Is_laundering'] == 0.0]
fraudulent_transactions = df[df['Is_laundering'] == 1.0]

print(f"Total number of legit transactions: {format_number(len(legit_transactions))}")
print(f"Total number of fraudulent transactions: {format_number(len(fraudulent_transactions))}")

Total number of legit transactions: 8,745,523
Total number of fraudulent transactions: 8,962


In [13]:
transactions_per_method = df['Payment_type'].value_counts().apply(lambda x: f'{x:,}')

print(f"Transactions per method: {transactions_per_method}")

Transactions per method: Payment_type
Credit card        1,853,835
Debit card         1,853,199
Cheque             1,852,629
ACH                1,850,212
Cross-border         859,452
Cash Withdrawal      276,990
Cash Deposit         208,168
Name: count, dtype: object


In [14]:
suspicious_transactions_mean_amount_usd = df[df['Is_laundering'] == 1.0]['Amount_usd'].mean()
legit_transactions_mean_amount_usd = df[df['Is_laundering'] == 0.0]['Amount_usd'].mean()

print(f"Mean amount (USD) for suspicious transactions: {suspicious_transactions_mean_amount_usd:,.2f}")
print(f"Mean amount (USD) for legit transactions: {legit_transactions_mean_amount_usd:,.2f}")

Mean amount (USD) for suspicious transactions: 41,302.69
Mean amount (USD) for legit transactions: 8,736.88


In [15]:
suspicious_transactions_coming = df[df['Is_laundering'] == 1.0]['Sender_bank_location'].value_counts()

print(f"Suspicious transactions coming from: {suspicious_transactions_coming}")

Suspicious transactions coming from: Sender_bank_location
UK             8389
Morocco          48
Mexico           43
India            41
Switzerland      40
Germany          38
Italy            36
Japan            35
Netherlands      35
Austria          34
UAE              34
USA              33
France           28
Albania          27
Turkey           27
Spain            26
Pakistan         25
Nigeria          23
Name: count, dtype: int64


In [16]:
most_suspicious_methods = df[df['Is_laundering'] == 1.0]['Payment_type'].value_counts()

print(f"Most suspicious methods: {most_suspicious_methods}")

Most suspicious methods: Payment_type
Cross-border       2394
Cash Deposit       1282
Cash Withdrawal    1195
ACH                1063
Credit card        1029
Debit card         1007
Cheque              992
Name: count, dtype: int64


In [17]:
relation_between_methods = df.groupby('Payment_type')['Is_laundering'].mean()

print(f"Relation between methods: {relation_between_methods}")

Relation between methods: Payment_type
ACH                0.000575
Cash Deposit       0.006158
Cash Withdrawal    0.004314
Cheque             0.000535
Credit card        0.000555
Cross-border       0.002785
Debit card         0.000543
Name: Is_laundering, dtype: float64
